Índice

1. Estrutura básica
2. Declaração return
3. Funções compactas
4. Declarações de tipos
   - Tipos abstratos e concretos
5. Retorno de múltiplos valores
6. Funções variádicas
7. Argumentos nomeados
8. Funções anônimas
9. Funções mutáveis e não-mutáveis
10. Funções de alto nível
11. Duck-typing

# Funções

Em Julia, uma função é um objeto que mapeia uma tupla de valores de argumentos para retornar um (ou mais) valores.

In [1]:
# A estrutura básica de uma função
function uma_funcao(arg1, arg2)
    resultado = arg1 + arg2
    return resultado
end

# invocação de uma função não difere de outras linguagens
uma_funcao(1, 2)

3

## Return

Em Julia, o último valor avaliado dentro da função será, por padrão, retornado. Contudo, assim como em muitas outras linguagens, a declaração ```return``` pode ser usada para fazer com que a função retorne um valor de forma imediata. Em outras palavras, o uso é opcional.

## Funções compactas
Funções simples podem ser reescritas de forma compacta:

In [2]:
# nesta construção o return também é opcional
uma_funcao_compacta(arg1, arg2) = return arg1 + arg2

uma_funcao_compacta(1, 2)

3

## Tipos e declarações de tipos

Julia é dinâmica. Dessa forma, a linguagem é capaz de inferir os tipos usados ao decorrer do código sem que eles tenham sido explicitamente declarados. Apesar disso, é possível declarar tipos para argumentos e retornos de funções.

Há dois motivos principais para isso:
- Emitir um ```assert``` e garantir que um programa funcione como esperado (um erro será levantado caso a anotação não seja condizente).
- Oferecer informação extra sobre tipos ao compilador, pois, em alguns casos, isso pode otimizar o desempenho.

Para maiores detalhes, veja a [documentação oficial](https://docs.julialang.org/en/v1/manual/types/#Type-Declarations).


**Sobre anotações de tipo em Jupyter**

Aparentemente, após a primeira compilação com anotações de tipo, é necessário limpar o Kernel caso as anotações sejam alteradas. Do contrário, a versão pré-compilada será usada e o comportamento pode não ser o esperado.

In [3]:
# este exemplo levantará um TypeError
(1 + 2)::AbstractFloat

LoadError: TypeError: in typeassert, expected AbstractFloat, got a value of type Int64

In [10]:
# este exemplo não levantará um erro
(1 + 2)::Int64

3

A anotação de tipos em funções é feita após a delimitação dos argumentos.

In [5]:
function g(x, y)::Int8
    return x * y
end

typeof(g(1, 2))

Int8

Naturalmente, não é apenas possível anotar o tipo retorno de uma função, mas também os próprios argumentos.

In [6]:
# trocar a declaração de Float64 como retorno da função levantaria um InexactError
# trocar a declaração de um dos argumentos levaria a um MethodError
function j(x::Float64, y::Int64)::Float64
    return x ^ y
end

j(2.5, 2)

6.25

Uma peculiaridade de Julia é possibilitar a definição de comportamentos distintos para uma função de mesmo nome a depender do tipo dos argumentos.

In [7]:
function arredondar(x::Float64)
    return round(x)
end

function arredondar(x::Int64)
    return x
end

methods(arredondar)

# 2 methods for generic function "arredondar":
[1] arredondar(x::Float64) in Main at In[7]:1
[2] arredondar(x::Int64) in Main at In[7]:5

É possível utilizar tipos abstratos para anotação.

Se a ideia ao declarar uma função for definir uma função que consiga lidar com diferentes tipos de um mesmo grupo de tipos, podemos utilizar um tipo abstrato como ```AbstractFloat``` ou um ```Integer```. Uma lista completa dos tipos de inteiros e floats pode ser encontrada na [documentação oficial](https://docs.julialang.org/en/v1/manual/integers-and-floating-point-numbers/#Integers-and-Floating-Point-Numbers). 

Maiores explicações sobre tipos abstratos poderão ser encontradas mais a frente.

In [ ]:
# esta função lidará com todos os tipos de floats
function funcao1(x::AbstractFloat)
    return round(x)
end

# esta função poderá receber todos os tipos de inteiros
function funcao2(x::Integer)
    return round(x)
end

### Mais sobre tipos: abstratos e concretos 

Tipos abstratos são nós na hierarquia de tipos da linguagem. A partir de tipos abstratos, tipos concretos podem ser definidos. Dessa forma, tipos concretos sempre são subtipos de tipos abstratos.

A síntaxe geral para declarar tipos abstratos é:

```julia
abstract type «name» end
abstract type «name» <: «supertype» end
```

A declaração ```abstract``` introduz um novo tipo abstrato, cujo nome é dado por «name». Esse nome pode ser opcionalmente seguido por ```<:``` e um nome de tipo abstrato já existente, indicando que o tipo recém declarado é subtipo de um tipo parental. Se nenhum «supertype» é dado, ele será ```Any```. Any é o topo da árvore de tipos em Julia, em oposição a ```Union{}``` que representa o fim da árvore. Para maiores detalhes, veja a [documentação oficial](https://docs.julialang.org/en/v1/manual/types/#man-abstract-types).

Abaixo, temos a relação de tipos abstratos que definem a hierarquia numérica da linguagem.

```julia
abstract type Number end
abstract type Real <: Number end
abstract type AbstractFloat <: Real end
abstract type Integer <: Real end
abstract type Signed <: Integer end
abstract type Unsigned <: Integer end
```

## Retorno de múltiplos valores

## Funções variádicas (Varargs)

Uma função variádica é uma função de aridez indefinida. Ou seja: que aceita um número variável de argumentos. Em Julia, podemos definir N parâmetros acrescentando ```...``` (elipse) após o último argumento.

In [45]:
function vararg(x...)
    for arg in x
        print("$(arg * 2) ")
    end
end;

In [46]:
vararg(2, 3, 4)  # 4 6 8

4 6 8 

## Argumentos nomeados

Algumas funções podem aceitar argumentos nomeados ao invés de argumentos posicionais. Esses argumentos funcionam como argumentos regulares, exceto por serem definidos após os argumentos regulares da função e serem separados por ponto-e-vírgula (;). Outra diferença é que argumentos nomeados devem sempre possuir um valor padrão na declaração da função.

Por exemplo, vamos definir uma função que calcula o valor de log (com base 2.718281828459045).

Note que as anotações indicam tipo Real, dessa forma cobrindo todos os tipos derivados de Integer e AbstractFloat, pois os mesmos são subtipos de Real.

In [32]:
AbstractFloat <: Real && Integer <: Real

true

In [33]:
function logaritmo(x::Real; base::Real=2.7182818284590)
    return log(base, x)
end

logaritmo (generic function with 1 method)

In [34]:
# função irá funcionar sem que o argumento base seja especificado
logaritmo(10)

2.3025850929940845

In [35]:
# também funcionará com um valor diferente para o parâmetro base
logaritmo(10; base=2)

3.3219280948873626

## Funções anônimas

Frequentemente funções são necessárias, mas não necessariamente precisam de um nome. Para esses casos, usamos funções anônimas.

A síntaxe é construída da seguinte forma: utilizamos o operador ```->```. À esquerda de -> definimos o nome do parâmetro da função. À direita de -> definimos a operação que será aplicada sobre o parâmetro que definimos à esquerda. No exemplo abaixo, iremos aplicar exponenciação a um valor log para desfazer a transformação logaritmica.

In [37]:
map(x -> 2.7182818284590 ^ x, logaritmo(2))

2.0

## Funções mutáveis e não-mutáveis

Por convenção, funções seguidas de ```!``` alteram o conteúdo do objeto recebido, enquantos funções sem ! não. Por exemplo:

In [24]:
v = [3, 5, 2];

In [25]:
sort(v)

3-element Vector{Int64}:
 2
 3
 5

In [27]:
# v não foi alterado
v

3-element Vector{Int64}:
 3
 5
 2

In [29]:
sort!(v);

In [30]:
# agora v foi alterado
v

3-element Vector{Int64}:
 2
 3
 5

## Funções de alto nível

## Duck-typing

Funções em Julia funcionarão sempre que as entradas fizerem sentido com as operações envolvidas na função.

In [11]:
function f(x)
    x^2
end

f (generic function with 1 method)

In [12]:
# naturalmente, f funcionará em números
f(42)

1764

In [14]:
# f funcionará em matrizes
A = rand(3, 3)
f(A)

3×3 Matrix{Float64}:
 0.507536  0.967503  1.21788
 0.243742  0.52587   0.488928
 0.504484  1.16218   1.58205

In [19]:
# f funcionará em valores string, pois o operador * causa concatenação.
f("olá")

"oláolá"